In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/usr/local/python/lib/python3.10/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


## Download data

In [2]:
ejesviales = gpd.read_file('../../data/external/temporal_todocker/0_marcogeo_ejesviales_prueba.gpkg')
hubdistance = gpd.read_file('../../data/external/temporal_todocker/02_hubdistance.gpkg')

In [3]:
print(ejesviales.shape)
ejesviales.head(1)

(768, 13)


,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVEVIAL,CVESEG,NOMVIAL,TIPOVIAL,SENTIDO,TIPOSEN,AMBITO,CVEGEOSEG,geometry
0,140390001,14,039,0001,00184,00003,Teatro de la República,Calle,Un sentido,1,Urbana,1403900010018400003,"MULTILINESTRING ((666274.061 2287716.662, 6662..."


In [4]:
print(hubdistance.shape)
hubdistance.head(2)

(23044, 15)


,fid,cat,Tipo_de_se,Clasificac,Clasifica0,idruta,fid_2,cat_2,Tipo_de_se_2,Clasificac_2,Clasifica0_2,idruta_2,HubName,HubDist,geometry
0,1,1,Complementaria,Rutas complementarias y otros servicios,Operando,34,2,1,Complementaria,Rutas complementarias y otros servicios,Operando,34,1412000010358500034,2.604311,POINT (665756.141 2287120.589)
1,1,1,Complementaria,Rutas complementarias y otros servicios,Operando,34,165,2,Complementaria,Rutas complementarias y otros servicios,Operando,36,1412000010358500034,2.604311,POINT (665756.141 2287120.589)


In [5]:
columns_tokeep = ['idruta','HubName','geometry']
hubdistance_f = hubdistance[columns_tokeep]
hubdistance_f = hubdistance_f.rename(columns={'HubName':'segmentocalle'})
hubdistance_f.head(1)

,idruta,segmentocalle,geometry
0,34,1412000010358500034,POINT (665756.141 2287120.589)


In [6]:
rutas = list(hubdistance_f.idruta.unique())
print(len(rutas))

segmentos = list(hubdistance_f.segmentocalle.unique())
print(len(segmentos))

26
138


## Prueba separando rutas y group by segmentos

In [29]:
gdf_ruta_prueba = hubdistance_f[hubdistance_f.idruta == 34]

#Contar las veces que la ruta pasa por un segmento de calle
segs_deruta = gdf_ruta_prueba.groupby('segmentocalle').count()

#Formato
columns_tokeep = ['idruta']
segs_deruta = segs_deruta[columns_tokeep]
segs_deruta = segs_deruta.reset_index()
segs_deruta = segs_deruta.rename(columns={'idruta':'count'})

segs_deruta

,segmentocalle,count
0,1403900010015900003,4
1,1403900010017200002,4
2,1403900010315600046,4
3,1403900010434500003,1
4,1403900010863100001,60
5,1403900010863100004,4
6,1403900010863100005,8
7,1403900010863100015,8
8,1403900010863100017,16
9,1403900010863100018,28


In [65]:
ruta = 34

dicc_segmento_prueba = {}
listavacia = []

for row_id,row_series in segs_deruta.iterrows():
    segmento_calle = row_series[0]
    count = row_series[1]
    
    if count > 1: #Si la ruta pasa solo 1 vez, probablemente es un segmento en una intersección y la ruta no usa por esa calle.
        if segmento_calle not in dicc_segmento_prueba:
            dicc_segmento_prueba[segmento_calle] = listavacia
    
        if ruta not in dicc_segmento_prueba[segmento_calle]:
            dicc_segmento_prueba[segmento_calle].append(ruta)

In [66]:
dicc_segmento_prueba

{'1403900010015900003': [34],
 '1403900010017200002': [34],
 '1403900010315600046': [34],
 '1403900010863100001': [34],
 '1403900010863100004': [34],
 '1403900010863100005': [34],
 '1403900010863100015': [34],
 '1403900010863100017': [34],
 '1403900010863100018': [34],
 '1403900010863100019': [34],
 '1403900010863100020': [34],
 '1412000010358500031': [34],
 '1412000010358500034': [34],
 '1412000010358500036': [34],
 '1412000011048800002': [34]}

## Prueba con un solo segmento

In [169]:
#Por el segmento 1412000011131900003 solo pasan la ruta 70 y 71

In [185]:
gdf_segmento_prueba = hubdistance[hubdistance.HubName == '1412000011131900003']

dicc_segmento_prueba = {}
listavacia = []

for index,row in gdf_segmento_prueba.iterrows():
    segmento_calle = row.HubName #Es un str   
    ruta = row.idruta #Es un int
    
    if segmento_calle not in dicc_segmento_prueba:
        dicc_segmento_prueba[segmento_calle] = listavacia
    
    if ruta not in dicc_segmento_prueba[segmento_calle]:
        dicc_segmento_prueba[segmento_calle].append(ruta)

In [186]:
dicc_segmento_prueba

{'1412000011131900003': [70, 71]}

## Prueba iterando en los segmentos

In [190]:
segmentos = list(hubdistance.HubName.unique())

dicc_segmentos_rutas = {}
listavacia = []

for segmento in segmentos:
    gdf_segmento_actual = hubdistance[hubdistance.HubName == segmento]
    
    for index,row in gdf_segmento_actual.iterrows():
        segmento_calle = row.HubName #Es un str
        ruta = row.idruta #Es un int

        if segmento_calle not in dicc_segmentos_rutas:
            dicc_segmentos_rutas[segmento_calle] = listavacia

        if ruta not in dicc_segmentos_rutas[segmento_calle]:
            dicc_segmentos_rutas[segmento_calle].append(ruta)    

In [191]:
dicc_segmentos_rutas['1412000011131900003']

[34,
 36,
 42,
 43,
 44,
 45,
 46,
 47,
 72,
 73,
 167,
 168,
 173,
 174,
 39,
 40,
 103,
 104,
 107,
 108,
 109,
 110,
 149,
 150,
 70,
 71]

## Resto del código

In [161]:
rutas = list(hubdistance.idruta.unique())

dicc_segmentos_rutas = {}
listavacia = []

for ruta in rutas:
    gdf_ruta_actual = hubdistance[hubdistance.idruta == ruta]
    
    for index,row in gdf_ruta_actual.iterrows():
        segmento_calle = row.HubName
        
        #Si en esta ruta no había salido este segmento, añadirlo
        if segmento_calle not in dicc_segmentos_rutas:
            dicc_segmentos_rutas[segmento_calle] = listavacia
        
        #Agregar la ruta a ese segmento, si no estaba antes
        if ruta not in dicc_segmentos_rutas[segmento_calle]:
            dicc_segmentos_rutas[segmento_calle].append(ruta)

In [164]:
print(len(segmentos_rutas))

138


In [ ]:
for segmento in segmentos_rutas:
    print(len(segmento))

## Iteración fallida

In [69]:
hubdistance.head(2)

,fid,cat,Tipo_de_se,Clasificac,Clasifica0,idruta,fid_2,cat_2,Tipo_de_se_2,Clasificac_2,Clasifica0_2,idruta_2,HubName,HubDist,geometry
0,1,1,Complementaria,Rutas complementarias y otros servicios,Operando,34,2,1,Complementaria,Rutas complementarias y otros servicios,Operando,34,1412000010358500034,2.604311,POINT (665756.141 2287120.589)
1,1,1,Complementaria,Rutas complementarias y otros servicios,Operando,34,165,2,Complementaria,Rutas complementarias y otros servicios,Operando,36,1412000010358500034,2.604311,POINT (665756.141 2287120.589)


In [77]:
## Diccionario de segmentos que tienen rutas y las rutas que tienen
dicc_segmentos_rutas = {}
listavacia = []

#Crear un diccionario con los segmentos de calle y una lista de rutas por segmento
for index,row in hubdistance.iterrows():    
    segmento_calle = row[12] #Es un str   
    ruta = row[5] #Es un int
    
    #Si es la primera vez que encuentra este segmento, agregarlo con una lista vacía
    if segmento_calle not in dicc_segmentos_rutas:
        dicc_segmentos_rutas[segmento_calle] = listavacia

    #Para esta row en la que iteramos (segmento de calle y ruta), si es la primera vez que encuentra la ruta en este segmento, agregarla a esa lista:
    if ruta not in dicc_segmentos_rutas[segmento_calle]:
        dicc_segmentos_rutas[segmento_calle].append(ruta)

In [79]:
#dicc_segmentos_rutas

In [80]:
for segmento in dicc_segmentos_rutas:
    print(len(segmento))

19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
